In [1]:
# Redmine ticket 6885 - Removing a panel from a visit
import os
import sys

import django

sys.path.append('../..')  # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

  * Reading config from flourish.ini
Loading Data Encryption (init)...
 * loading keys from /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields
 * loading rsa.restricted.public ... Done.
 * loading rsa.restricted.private ... Done.
 * loading rsa.local.public ... Done.
 * loading rsa.local.private ... Done.
 * loading aes.local ... Done.
 * loading aes.restricted ... Done.
 * loading salt.local ... Done.
 * loading salt.restricted ... Done.
 Done loading Data Encryption (init)...


/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/django_q/conf.py:139: UserWarning: Retry and timeout are misconfigured. Set retry larger than timeout, 
        failure to do so will cause the tasks to be retriggered before completion. 
        See https://django-q.readthedocs.io/en/latest/configure.html#retry for details.
  warn(
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (HistoryManagerMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (ScheduleModelMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (VisitModelMixin) without inherit=True
  warnings.wa

Loading Data Encryption ...
 * found encryption keys in /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields.
 * using model django_crypto_fields.crypt.
 Done loading Data Encryption.
Loading Edc Consent ...
 * checking for site consents ...
 * registered consents 'consents' from 'flourish_caregiver'
 * registered consents 'consents' from 'flourish_child'
 * registered consents 'consents' from 'pre_flourish'
 * registered consents 'consents' from 'flourish_facet'
 * flourish_caregiver.subjectconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 2 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 3 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 4 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbadolconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbinformedconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_child.childdum

In [7]:
from collections import Counter
from django.db.models import Q


def get_inconsistent_menarche_start_dt_objects(participant_identifier):
    """
    Identify all objects of a participant whose `menarche_start_dt` 
    differs from the majority.

    Args:
    - participant_identifier (str): The identifier for the participant.

    Returns:
    - List of objects that have a different `menarche_start_dt` than the majority.
    - The majority `menarche_start_dt`.
    """
    # Replace `YourModel` with the actual model name that extends `PregTestModelMixin`
    participant_objects = ChildPregTesting.objects.filter(
        child_visit__subject_identifier=participant_identifier, menarche_start_dt__isnull=False)

    if not participant_objects.exists():
        return [], None

    menarche_start_dt_counter = Counter()

    # Count occurrences of each `menarche_start_dt`
    for obj in participant_objects:
        menarche_start_dt_counter[obj.menarche_start_dt] += 1

    # Identify the majority `menarche_start_dt`
    majority_date, _ = menarche_start_dt_counter.most_common(1)[0]

    # Find objects that differ from the majority date
    inconsistent_objects = [
        obj for obj in participant_objects if obj.menarche_start_dt != majority_date
    ]

    return inconsistent_objects, majority_date





In [14]:
import pandas as pd

from flourish_child.models import ChildPregTesting

# Get unique subject identifiers
adolocents = list(set(ChildPregTesting.objects.all().values_list('child_visit__subject_identifier', flat=True)))

# Initialize list to store the data for the DataFrame
data = []

for child in adolocents:
    inconsistent_objects, majority_date = get_inconsistent_menarche_start_dt_objects(child)
    if inconsistent_objects:
        for obj in inconsistent_objects:
            data.append([
                child,
                majority_date,
                len(inconsistent_objects),  # Number of inconsistent objects
                obj.child_visit.visit_code,
                obj.menarche_start_dt
            ])

# Create DataFrame
df = pd.DataFrame(data, columns=[
    'Child', 
    'Majority Menarche Start Date', 
    'Inconsistent Objects',  # Add a column for the number of inconsistent objects
    'Inconsistent Object Visit Code', 
    'Inconsistent Object Menarche Start Date'
])

# Export the DataFrame to a CSV file
df.to_csv('inconsistent_menarche_start_dt.csv', index=False)
